In [1]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%pip install \
--trusted-host 192.168.210.244 \
--index-url http://192.168.210.244:8081/repository/pypi/simple/ \
snowflake-connector-python

In [3]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZPRbtowFIZfJfKukzgOpcFqqAIIka0drMA07S44J2Dh2JntkPL2M6FI3UV7szvL%2Fo79%2Bfz2w%2BNrLbwTaMOVTFEUYOSBZKrkcp%2Bi7WbuJ8gztpBlIZSEFJ3BoMfxgylq0dCstQf5An9aMNZzG0lD%2B4UUtVpSVRhuqCxqMNQyus6enygJMG20soopgd6VfF5RGAPaOsNbSWm40ztY29Aw7Lou6OJA6X1IMMYhHoWOuiBfbvyru9MHfBTiwYV3hMNXb24TLq8t%2BExrd4UMXWw2K3%2B1XG%2BQl91Up0qatga9Bn3iDLYvT1cB4wwaPRiNMAmk0vbgQ6tVA0Gj%2BamwILg8BkaqrhLFEZiqm9a6QwI3CisoQ6H23PUhn6WoOfLysJraEzTftpNs2dbL6eLHc74gC03Ok1%2Fbr%2Fd8KpLcZEU07xKGvJ%2B3oMkl6NyYFnJ5ide6KUxiHxOfkE2EKRnSuziIyP1v5M1cvFwWtq%2B83aH3CGrOtDKqsk

('7.6.1', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

In [4]:
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.DEMOGRAPHIC
'''

cur = ctx.cursor().execute(query)
demographic_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

Problem dummies

In [5]:
# Import
path = r'data/problem_charlson_score.csv'
problem_charlson_score = pd.read_csv(path)

In [25]:
# Merge
problem_charlson = pd.merge(problem_charlson_score, demographic_df[['SUBJECT', 'DEATH_DATE']], how='left')

In [28]:
# To datetime
problem_charlson['DT_TM'] = pd.to_datetime(problem_charlson['DT_TM']).dt.date
problem_charlson['DEATH_DATE'] = pd.to_datetime(problem_charlson['DEATH_DATE']).dt.date

In [ ]:
# Work out if death within a year
def year_fun(row):
    if pd.notnull(row['DEATH_DATE']):
        if row['DEATH_DATE'] <= row['DT_TM'] + pd.offsets.DateOffset(years=1):
            return 1

In [47]:
# Work out if death at anytime
def death_fun(row):
    if pd.notnull(row['DEATH_DATE']):
        return 1

In [ ]:
# Apply
problem_charlson['death_year_label'] = problem_charlson.apply (lambda row: year_fun (row),axis=1)    
problem_charlson['death_label'] = problem_charlson.apply (lambda row: death_fun (row),axis=1) 

In [54]:
# save
#problem_charlson.to_csv('problem_charlson_death.csv', index=False)

MED

In [5]:
# Import
path = r'data/problem_charlson_score_med.csv'
problem_charlson_score_med = pd.read_csv(path)

In [ ]:
problem_charlson_med = pd.merge(problem_charlson_score_med, demographic_df[['SUBJECT', 'DEATH_DATE']], how='left')
problem_charlson_med['DT_TM'] = pd.to_datetime(problem_charlson_med['DT_TM']).dt.date
problem_charlson_med['DEATH_DATE'] = pd.to_datetime(problem_charlson_med['DEATH_DATE']).dt.date
problem_charlson_med['death_year_label'] = problem_charlson_med.apply (lambda row: year_fun (row),axis=1)    
problem_charlson_med['death_label'] = problem_charlson_med.apply (lambda row: death_fun (row),axis=1) 

In [13]:
# Save
#problem_charlson_med.to_csv('problem_charlson_death_med.csv', index=False)

Med fuzzy

In [15]:
# Import
path = r'data/problem_charlson_score_med_fuzzy.csv'
problem_charlson_score_med_fuzzy = pd.read_csv(path)

In [ ]:
problem_charlson_med_fuzzy = pd.merge(problem_charlson_score_med_fuzzy, demographic_df[['SUBJECT', 'DEATH_DATE']], how='left')
problem_charlson_med_fuzzy['DT_TM'] = pd.to_datetime(problem_charlson_med_fuzzy['DT_TM']).dt.date
problem_charlson_med_fuzzy['DEATH_DATE'] = pd.to_datetime(problem_charlson_med_fuzzy['DEATH_DATE']).dt.date
problem_charlson_med_fuzzy['death_year_label'] = problem_charlson_med_fuzzy.apply (lambda row: year_fun (row),axis=1)    
problem_charlson_med_fuzzy['death_label'] = problem_charlson_med_fuzzy.apply (lambda row: death_fun (row),axis=1) 

In [18]:
# Save
#problem_charlson_med_fuzzy.to_csv('problem_charlson_death_med_fuzzy.csv', index=False)